In [1]:
from PIL import Image, ImageDraw 
import numpy as np
import matplotlib.pyplot as plt


def show_3_image(image):
    """Выводим изображние + 3 его составляющие"""
    fig=plt.figure(figsize=(15, 15))
    fig.add_subplot(1, 4, 1)
    plt.imshow(image)
    fig.add_subplot(1, 4, 2)
    plt.imshow(image[:,:,0])
    fig.add_subplot(1, 4, 3)
    plt.imshow(image[:,:,1])
    fig.add_subplot(1, 4, 4)
    plt.imshow(image[:,:,2])
    plt.show()
    
    
def binary_image(image, limit, color):
    """Бинаризация изображения
    limit - пороговое значение пикселя
    color - цвет для порогового значения 'r', 'g', 'b'
    """
    result_image = Image.fromarray(image)
    width = image.shape[1]
    height = image.shape[0]
    pix = Image.fromarray(image).load() # значения пикселей
    draw = ImageDraw.Draw(result_image) #Создаем инструмент для рисования
    for i in range(width):
        for j in range(height):
            r = pix[i, j][0]
            g = pix[i, j][1]
            b = pix[i, j][2]
            if (color=='r' and r > limit):
                r, g, b = 255, 255, 255
            elif (color=='g' and g > limit):
                r, g, b = 255, 255, 255
            elif(color=='b' and b > limit):
                r, g, b = 255, 255, 255
            else:
                r, g, b = 0, 0, 0                
            draw.point((i, j), (r, g, b))
    return result_image

def binary_image_by_summary(image, limit):
    """Бинаризация изображения
    limit - пороговое значение пикселя g+b
    """
    result_image = Image.fromarray(image)
    width = image.shape[1]
    height = image.shape[0]
    pix = Image.fromarray(image).load() # значения пикселей
    draw = ImageDraw.Draw(result_image) #Создаем инструмент для рисования
    for i in range(width):
        for j in range(height):
            r = pix[i, j][0]
            g = pix[i, j][1]
            b = pix[i, j][2]
            if (g+b > 2*limit):
                r, g, b = 255, 255, 255
            else:
                r, g, b = 0, 0, 0                
            draw.point((i, j), (r, g, b))
    return result_image


def clear_image(image, limit_pixels):
    """Удаляет шум"""
    image = np.array(image)
    result_image = Image.fromarray(image)
    draw = ImageDraw.Draw(result_image)
    pix = result_image.load() 
    width = image.shape[1]
    height = image.shape[0]
    for i in range(10, width-10):
        for j in range(10, height-10):
            # Рамка 9*9 - смотрим чему равны значения по периметру и закрашиваем все внутри также
            frame = np.array([])
            for k in range(i-4, i+5):  # верхняя рамка
                frame = np.append(frame, pix[k, j-4][0])
            for k in range(i-4, i+5):  # нижняя рамка
                frame = np.append(frame, pix[k, j+5][0])
            for k in range(j-4, j+5):  # левая рамка
                frame = np.append(frame, pix[i-4, k][0])
            for k in range(j-4, j+5):  # правая рамка
                frame = np.append(frame, pix[i+5, k][0])
            unique, counts = np.unique(frame, return_counts=True)
            counters = dict(zip(unique, counts))
            count_white = counters.get(1)
            count_black = counters.get(0)
            if (not not count_white and count_white > limit_pixels and pix[i, j][0] != 255):
                for k1 in range(i-4, i+5):
                    for k2 in range(j-4, j+5):
                        draw.point((k1, k2), (255, 255, 255)) 
            if (not not count_black and count_black > limit_pixels and pix[i, j][0] != 0):
                for k1 in range(i-4, i+5):
                    for k2 in range(j-4, j+5):
                        draw.point((k1, k2), (0, 0, 0))
    return result_image

def find_puzzlers(image, x_frame, y_frame, limit_pixels):
    """Возвращает найденные пазлы
        x_frame/2, y_frame/2 - рамка соответствующая размерам пазлов
    """
    puzzle_count = 0
    puzzle_position_dict = dict()  # key-номер пазла, value-((x_center, y_center), radius)
    image_np = np.array(image)
    result_image = Image.fromarray(image_np)  # Тут закрашиваем найденные пазлы черным
    draw = ImageDraw.Draw(result_image)
    return_image = Image.fromarray(image_np)  # Тут обводим пазлы окружностью
    draw_result = ImageDraw.Draw(return_image)
    pix = result_image.load() 
    width = image_np.shape[1]
    height = image_np.shape[0]
    for i in range(x_frame, width-x_frame, int(x_frame/4)):
        for j in range(y_frame, height-y_frame, int(y_frame/4)):
            frame = np.array([])  # рамка x_frame(столбцов)*y_frame(строк) для нахождения пазлов
            # заполняем рамку всеми пикселями внутри нее
            for k1 in range(i-x_frame, i+x_frame):
                for k2 in range(j-y_frame, j+y_frame):
                    frame = np.append(frame, pix[k1, k2][0])
            # подсчитываем сколько белых пикселей внутри рамки
            unique, counts = np.unique(frame, return_counts=True)
            counters = dict(zip(unique, counts))
            count_white = counters.get(255)
            if (not not count_white and count_white > limit_pixels):
                # находим более оптимальное положение пазла
                optim_i = i
                optim_j = j
                optim_radius = int(x_frame)
                max_white_count = count_white
                for o1 in range(0, int(x_frame/2), int(x_frame/8)): # двигаем немного вправо
                    for o2 in range(0, int(y_frame/2), int(y_frame/8)):  #двигаем немного вниз
                        for o3 in range(0, 6, 1):  #немного увеличиваем радиус рамки 
                            frame = np.array([])
                            for k1 in range((i+o1)-(x_frame+o3), (i+o1)+(x_frame+o3)):
                                for k2 in range((j+o2)-(y_frame+o3), (j+o2)+(y_frame+o3)):
                                    frame = np.append(frame, pix[k1, k2][0])
                            unique, counts = np.unique(frame, return_counts=True)
                            counters = dict(zip(unique, counts))
                            count_white = counters.get(255)
                            # нашли позицию рамки лучше
                            if (count_white > max_white_count):
                                max_white_count = count_white
                                optim_i = i+o1
                                optim_j = j+o2
                                optim_radius = int(x_frame+o3)
                print('puzzle found i=', i, "j =", j, 'radius =', int(x_frame), '   optim_i=', optim_i, "optim_j =", optim_j, 'optim_radius =', optim_radius)
                 # нашли пазл, найдем его координаты центра и радиус 
                puzzle_count = puzzle_count + 1
                radius = optim_radius
                puzzle_position_dict[puzzle_count] = ((optim_i, optim_j), radius, optim_radius)
                # У исходного изображения закрасим наденный пазл в черный цвет
                for k1 in range(optim_i-x_frame, optim_i+x_frame):
                    for k2 in range(optim_j-y_frame, optim_j+y_frame):
                        draw.point((k1, k2), (0, 0, 0))
                # Обведем окружностью найденный пазл
                leftUpPoint = (optim_i-radius, optim_j-radius)
                rightDownPoint = (optim_i+radius, optim_j+radius)
                twoPointList = [leftUpPoint, rightDownPoint]
                draw_result.ellipse(twoPointList, outline=(255, 0, 0))
                radius = int(x_frame)+1
                leftUpPoint = (optim_i-radius, optim_j-radius)
                rightDownPoint = (optim_i+radius, optim_j+radius)
                twoPointList = [leftUpPoint, rightDownPoint]
                draw_result.ellipse(twoPointList, outline=(255, 0, 0))
                radius = int(x_frame)+1
                leftUpPoint = (optim_i-radius, optim_j-radius)
                rightDownPoint = (optim_i+radius, optim_j+radius)
                twoPointList = [leftUpPoint, rightDownPoint]
                draw_result.ellipse(twoPointList, outline=(255, 0, 0))
                
    return return_image, puzzle_count, puzzle_position_dict